# Import

In [2]:
import requests
import time
import random
from bs4 import BeautifulSoup
import json
import calendar
import pandas as pd

# Thu thập dữ liệu

In [2]:
#Hàm tạo file Date.txt chứa tất cả ngày theo dạng yyyy/mm/dd của các năm trong list years truyền vào
def Create_Date(years):
    date_file = open('Date.txt', 'w')
    for year in years:
        date = 1
        month = 1
        while date != 32 & month != 12:
            year = str(year)
            date_file.write(year)
            date_file.write('/')
            if month < 10:
                date_file.write('0')
            month = str(month)
            date_file.write(month)
            date_file.write('/')
            if date < 10:
                date_file.write('0')
            date = str(date)
            date_file.write(date)
            date_file.write('\n')
            year = int(year)
            month = int(month)
            date = int(date)
            if (date == 31 and month == 1) or (date == 29 and month == 2) or (date == 31 and month == 3) or (date == 30 and month == 4) or (date == 31 and month == 5) or (date == 30 and month == 6) or (date == 31 and month == 7) or (date == 31 and month == 8) or (date == 30 and month == 9) or (date == 31 and month == 10) or (date == 30 and month == 11):
                date = 1
                month += 1
            else:
                if date == 28 and month == 2:
                    if calendar.isleap(year) == True:
                        date += 1
                    else:
                        date = 1
                        month += 1
                else:
                    date += 1
            if date == 32 and month == 12:
                break
    date_file.close()

In [3]:
year = [2016, 2017, 2018, 2019]
Create_Date(year)
city_id = 1252431 # Đây là chỉ số khu vực thành phố Hồ Chí Minh trên trang web
all_data = []

# Đọc file lịch chứa các ngày cần lấy thông tin
date_file = open('Date.txt', 'r')
list_date = date_file.readlines()
date_file.close()


#Lấy dữ liệu bằng phương pháp API 
for date in list_date:
    date = date.replace('\n', '') # Xoá các dấu xuống dòng của date để request không bị lỗi
    url = f'https://www.metaweather.com/api/location/{city_id}/{date}/' 
    try:
        r = requests.get(url).json()
        all_data.append(r[0])
    except Exception as e: Trả về mô tả lỗi nếu bị lỗi
        print(f'Error: {e}')
        break 
    

In [10]:
# Cho mỗi thuộc tính vào từng list
data_id = []
state_name = []
state_abbr = []
wind_direction_compass = []
created = []
applicable_date = []
min_temp = []
max_temp = []
the_temp = []
wind_speed = []
wind_direction = []
air_pressure = []
humidity = []
visibility = []
predictability = []
for weather_data in all_data:
    data_id.append(weather_data['id'])
    state_name.append(weather_data['weather_state_name'])
    state_abbr.append(weather_data['weather_state_abbr'])
    wind_direction_compass.append(weather_data['wind_direction_compass'])
    created.append(weather_data['created'])
    applicable_date.append(weather_data['applicable_date'])
    min_temp.append(weather_data['min_temp'])
    max_temp.append(weather_data['max_temp'])
    the_temp.append(weather_data['the_temp'])
    wind_speed.append(weather_data['wind_speed'])
    wind_direction.append(weather_data['wind_direction'])
    air_pressure.append(weather_data['air_pressure'])
    humidity.append(weather_data['humidity'])
    visibility.append(weather_data['visibility'])
    predictability.append(weather_data['predictability'])
    

In [16]:
#Tạo dictionary từ các list thuộc tính
data_dict = {'ID': data_id, 'State Name': state_name, 'State Abbreviation': state_abbr, 
             'Wind Direction': wind_direction_compass,'Created': created, 'Applicable Date': applicable_date, 
             'Min Temp': min_temp, 'Max Temp': max_temp, 'The Temp': the_temp,
             'Wind Speed': wind_speed, 'Wind Direction': wind_direction, 'Air Pressure': air_pressure, 
             'Humidity': humidity, 'Visibility': visibility, 'Predictability': predictability}

# Tạo dataframe từ dictionary
df = pd.DataFrame(data_dict)

# Ghi dataframe vào file csv
df.to_csv('Data.csv')

# Chia bộ train và bộ test

Ta chia bộ dữ liệu theo tỉ lệ 70/30 (70: Bộ train, 30: Bộ test)

In [22]:
percent = 70
data = pd.read_csv('Data.csv')
data.drop(data.columns[0],axis = 1,inplace = True)   # Xóa cột Unnamed vì lúc ghi file chưa để index = False
num_row_train = int(data.shape[0] * percent / 100)
data_train = data[:num_row_train]
data_test = data[num_row_train:data.shape[0]]
print('Train: ',data_train.shape)
print('Test: ',data_test.shape)
data_train.to_csv('train.csv',index = False)
data_test.to_csv('test.csv',index = False)

Train:  (1022, 14)
Test:  (439, 14)


,ID,State Name,State Abbreviation,Wind Direction,Created,Applicable Date,Min Temp,Max Temp,The Temp,Wind Speed,Air Pressure,Humidity,Visibility,Predictability
1022,5.351000e+15,Heavy Rain,hr,123.115823,2018-10-19T15:37:05.468689Z,10/19/2018,23.796667,27.403333,26.600,3.017761,1015.625,92.0,5.558476,77
1023,5.458570e+15,Showers,s,103.472139,2018-10-20T15:37:07.576680Z,10/20/2018,25.103333,29.353333,28.075,7.784333,1016.780,83.0,10.043844,73
1024,5.260580e+15,Light Rain,lr,167.020164,2018-10-21T15:37:08.183410Z,10/21/2018,25.533333,30.020000,29.325,4.425343,1017.800,78.0,8.728090,75
1025,6.636980e+15,Heavy Rain,hr,28.607075,2018-10-22T15:37:08.388407Z,10/22/2018,23.833333,29.776667,28.255,1.692237,1016.145,82.0,9.838791,77
1026,5.655890e+15,Light Rain,lr,192.468486,2018-10-23T15:37:05.697214Z,10/23/2018,24.006667,29.353333,27.700,2.805340,1015.780,78.0,9.629700,75
